# Combined the following:
- https://docs.fast.ai/tutorial.data.html#Text
- https://docs.fast.ai/text.html#Building-a-classifier

<br>
<br>
<br>
<br>

# Notes:

Fastai's text module has three steps:
1. Pre-process data
2. Fine-tune a pre-trained model
3. Create other models (e.g. classifiers) on top of the encoder of the fine-tuned model

<br>
<br>
According to: https://gilberttanner.com/blog/fastai-sentiment-analysis

Standard FastAI training pipeline
1. Finding the best learning rate
2. Training the top layers
3. Unfreezing all layers

<br>
<br>

Regarding `fit_one_cycle`

- https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html
- https://sgugger.github.io/the-1cycle-policy.html

<br>
<br>

According to: https://forums.fast.ai/t/why-do-we-need-to-unfreeze-the-learner-everytime-before-retarining-even-if-learn-fit-one-cycle-works-fine-without-learn-unfreeze/41614/5

- By default when you load the model from fastai library, it will have all the layers frozen
- `learn.fit_one_cycle(2)` Train only the last layer group i.e fully connected layers. Don’t train longer because you might overfit.
- Use lr_find() before fit_one_cycle() to get best suited learning rate for underlying data.
- `learn.unfreeze()` sets every layer group to trainable (i.e. requires_grad=True).
- Use discriminative learning rates i.e. parameter `max_lr`, which maintains a low learning rate for initial layers as they need lesser tuning and gradually increase learning rate for later layers which need higher tuning especially fully connected ones.

In [1]:
from fastai.text import *

In [2]:
imdb = untar_data(URLs.IMDB_SAMPLE)

In [3]:
imdb

PosixPath('/Users/p787144/.fastai/data/imdb_sample')

In [4]:
df = pd.read_csv(imdb/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Can also be done by reading from dataframe, according to https://gilberttanner.com/blog/fastai-sentiment-analysis

`train_df, valid_df = df.loc[:12000,:], df.loc[12000:,:]`

`data_lm = TextLMDataBunch.from_df(path, train_df, valid_df, text_cols=10, bs=32)`

### Prepare data for fine-tuning the language model (lm)

In [5]:
data_lm = (
    TextList
    .from_csv(imdb, 'texts.csv', cols='text')
    .split_by_rand_pct()
    .label_for_lm() # this does the tokenization and numericalization
    .databunch()
)

data_lm.save()

data.show_batch( ) shows the beginning of each sequence of text along the batch dimension (the target being to guess the next word).

You  may notice that there are quite a few strange tokens starting with xx. These are special FastAI tokens that have the following meanings:

- xxunk: Token used instead of unknown words (words not found in the vocabulary).
- xxbos: Beginning of a text.
- xxfld: Represents separate parts of your document (several columns in a dataframe) like headline, body, summary, etc.
- xxmaj: Indicates that the next word starts with a capital, e.g. “House” will be tokenized as “xxmaj house”.
- xxup: Indicates that next word is written in all caps, e.g. “WHY” will be tokenized as “xxup why ”.
- xxrep: Token indicates that a character is repeated n times, e.g. if you have  10 $ in a row it will be tokenized as “xxrep 10 $” (in general “xxrep n  {char}”)
- xxwrep: Indicates that a word is repeated n times.
- xxpad : Token used as padding (so every text has the same length)

In [6]:
data_lm.show_batch()

idx,text
0,"! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj xxunk - hollywood style . xxmaj xxunk ! xxbos xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is"
1,"wrong . xxmaj before xxmaj xxunk xxunk for "" xxmaj xxunk "" when they real winner should of been xxmaj xxunk xxmaj fiennes for "" xxmaj sunshine "" . xxmaj if you have n't seen this movie yet , watch it and you 'll agree . "" xxmaj eyes xxmaj wide xxmaj shut "" when released xxunk no nominations . xxmaj and as far as this year goes , well"
2,"- of - age story and a wonderful movie . xxbos xxmaj old xxmaj jane 's mannered tale seems very popular these days . i have lost count of the number of versions going around . xxmaj probably the reason is that her "" xxunk "" are our "" xxunk "" even at this late date . xxmaj this xxup tv mini - series gives it a mannered telling suitable"
3,"does have the specific xxmaj naschy - flick - charm , it sadly drags far too much and gets really , really dull in - between . xxmaj naschy stars as the evil xxmaj baron xxmaj xxunk xxmaj de xxmaj xxunk , who xxunk the people and uses black magic and bloody xxunk to stay in power . xxmaj when xxmaj xxunk de xxmaj xxunk ( xxmaj xxunk xxmaj xxunk"
4,"the race thing hit pretty close to home for me . xxbos xxmaj four stories written by xxmaj robert xxmaj xxunk about various people who live in a beautiful , old mansion and what happens to them . xxmaj the first has xxmaj xxunk xxmaj elliott as a xxunk who sees the killer he 's writing about come to life . xxmaj some xxunk moments and the twist at the"


### Prepare data for training the classification model (clas)

In [7]:
data_clas = (
    TextList
    .from_csv(imdb, 'texts.csv', cols='text', vocab=data_lm.train_ds.vocab)
    .split_from_df(col='is_valid')
    .label_from_df(cols='label')
    .databunch(bs=32)
)

In [8]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj xxunk ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj xxunk to xxunk . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj xxunk for weeks before i watched it . i xxunk a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the xxunk - up xxunk of xxmaj new xxmaj xxunk . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj xxunk",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk gaming experience in general . xxmaj with graphics",positive


<br>
<br>

# 1. Basic Method:
### 1.1 Fine-tune

I believe in below it doesn't matter to specify `learn.unfreeze()` or not

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM)
learn.fit_one_cycle(2, 1e-2)
learn.save('mini_train_lm')
learn.save_encoder('mini_train_encoder')

epoch,train_loss,valid_loss,accuracy,time
0,4.290680,3.779507,0.288058,04:11
1,4.003562,3.749203,0.291881,04:35


To evaluate your language model, you can run the Learner.predict method and specify the number of words you want it to guess.

`learn.predict("This is a review about", n_words=10)`

Or

In [11]:
learn.show_results()

text,target,pred
xxbos xxup in xxup cold xxup blood is xxunk directed and adapted by xxmaj richard xxmaj brooks . xxmaj however,", it 's also so xxunk on being realistic , it 's sometimes more xxunk than entertaining . xxmaj xxunk",", the is not a xxunk that the xxunk and that is not xxunk xxunk than the . xxmaj the"
xxmaj xxunk and display their racism when a big black guy xxunk them out . xxmaj xxunk xxmaj xxunk runs,into a xxunk to hide behind some xxunk xxunk and in a xxunk realistic moment a large snake is hiding,"a the xxunk and get his the xxunk . . xxunk a xxunk , manner . xxunk xxunk xxunk xxunk"
i must say that they did have nice camera movement . xxmaj too bad the lighting did n't xxunk it,". xxmaj the cover of the xxup dvd is nice , and that 's where it ends . xxmaj just",. xxmaj it acting is the movie dvd is a to but it 's a the 's . xxmaj it
"xxmaj so , xxmaj todd xxmaj sheets once stated that he considers his xxunk , shot - on - video","z - epic , xxmaj zombie xxmaj bloodbath to be his first feature film . xxmaj anyone who 's ever",", - xxmaj , a xxunk xxmaj noir , be a worst film - . xxmaj he who does not"
"this is one of those that you watch on late night xxup tv , perhaps on xxup usa , simply",because you can not get to sleep . xxmaj watch it if you want but do not expect too much,"because i 're see see a see with xxmaj it this , you 're to you n't see it much"


### 1.2 Build a classifier

Currently has AWD_LSTM, Transformer and TransformerXL

Regarding `slice()`

- If you pass `slice(start,end)` then the first group's learning rate is `start`, the last is `end`, and the remaining are evenly geometrically spaced.

- If you pass just `slice(end)` then the last group's learning rate is `end`, and all the other groups are `end/10`. For instance (for our learner that has 3 layer groups):

- `learn.lr_range(slice(1e-5,1e-3)), learn.lr_range(slice(1e-3))`
- `(array([1.e-05, 1.e-04, 1.e-03]), array([0.0001, 0.0001, 0.001 ]))`

In [15]:
learn = text_classifier_learner(data_clas, AWD_LSTM)
learn.load_encoder('mini_train_encoder')
learn.fit_one_cycle(2, slice(1e-3,1e-2))
learn.save('mini_train_clas')

epoch,train_loss,valid_loss,accuracy,time
0,0.659081,0.616443,0.720000,06:03
1,0.602128,0.543012,0.780000,05:48


In [16]:
learn.show_results()

text,target,prediction
"xxbos \n \n i 'm sure things did n't exactly go the same way in the real life of xxmaj homer xxmaj hickam as they did in the film adaptation of his book , xxmaj rocket xxmaj boys , but the movie "" xxmaj october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj",positive,positive
"xxbos xxmaj to review this movie , i without any doubt would have to quote that memorable scene in xxmaj tarantino 's "" xxmaj pulp xxmaj fiction "" ( xxunk ) when xxmaj jules and xxmaj vincent are talking about xxmaj mia xxmaj wallace and what she does for a living . xxmaj jules tells xxmaj vincent that the "" xxmaj only thing she did worthwhile was pilot "" .",negative,negative
"xxbos xxmaj how viewers react to this new "" adaption "" of xxmaj shirley xxmaj jackson 's book , which was promoted as xxup not being a remake of the original 1963 movie ( true enough ) , will be based , i suspect , on the following : those who were big fans of either the book or original movie are not going to think much of this one",negative,negative
"xxbos xxmaj the trouble with the book , "" xxmaj memoirs of a xxmaj geisha "" is that it had xxmaj japanese xxunk but underneath the xxunk it was all an xxmaj american man 's way of thinking . xxmaj reading the book is like watching a magnificent ballet with great music , sets , and costumes yet performed by xxunk animals dressed in those xxunk far from xxmaj japanese",negative,positive
"xxbos xxmaj bonanza had a great cast of wonderful actors . xxmaj xxunk xxmaj xxunk , xxmaj pernell xxmaj whitaker , xxmaj michael xxmaj xxunk , xxmaj dan xxmaj blocker , and even xxmaj guy xxmaj williams ( as the cousin who was brought in for several episodes during 1964 to replace xxmaj adam when he was leaving the series ) . xxmaj the cast had chemistry , and they",positive,positive


<br>
<br>

# 2. Better Method:
### 2.1 Fine-tune

In [17]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.117025,3.796515,0.288714,04:11


In [18]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.785347,3.723795,0.297098,04:26


In [19]:
learn.predict("This is a review about", n_words=10)

'This is a review about the South Africa world , with an average'

In [20]:
learn.save_encoder('mini_train_encoder_improved')

### 2.2 Build a classifier

In [21]:
learn = text_classifier_learner(data_clas, AWD_LSTM) #, drop_mult=0.5)
learn.load_encoder('mini_train_encoder_improved')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.658751,0.619423,0.755000,05:51


Unfreeze the model and fine-tune it.

In [22]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.565165,0.487276,0.780000,06:01


In [23]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.473279,0.439402,0.800000,07:03


In [24]:
learn.predict("This was a great movie!")

(Category positive, tensor(1), tensor([0.0487, 0.9513]))